## AI Research Assistant using Watsonx.ai, LangGraph, and ReAct

This notebook showcases a personal AI research assistant built using the **IBM Watsonx.ai platform**, **LangGraph framework**, and the **ReAct agent architecture**. The assistant automates the end-to-end process of discovering and summarizing recent research papers from top scientific conferences.

### 🔍 Key Features and Workflow:

* Accepts a user prompt describing a research topic.
* Uses **Watsonx.ai LLM** to identify the relevant research domain (e.g., NLP, ML).
* Searches online repositories like **arXiv** for recent, relevant papers.
* Returns the top three papers with abstracts.
* On request, summarizes the papers using **LLM summarization tools**.
* Built with **LangGraph**, enabling multi-step planning and tool usage via the **ReAct pattern**.
* Hosted and orchestrated via **Watsonx.ai** services.

This AI-powered workflow streamlines literature review and research exploration, making it easier to stay up-to-date with the latest scientific advancements.

![Agent Workflow](flow.png)



### 📦 Import Required Libraries and Frameworks

This section loads all necessary libraries to build and run the research assistant agent. It includes:

* **Watsonx.ai LLM integration** (`ChatWatsonx`)
* **LangChain and LangGraph tools** for agent orchestration and planning
* **Prompt and message handling** for ReAct-style interactions
* **arXiv API integration** for research paper retrieval
* **Structured tool support** via `Tool`, `StructuredTool`, and `BaseModel`
* **Environment variable loading** using `dotenv`
* **Markdown display** for nicer outputs in notebooks

These imports set the foundation for all subsequent components of the agent.


In [58]:
from langchain_ibm import ChatWatsonx
from dotenv import load_dotenv
import os
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from IPython.display import Markdown, display
import requests
import xml.etree.ElementTree as ET
from langchain.tools import Tool
from pydantic import BaseModel
from typing import Optional
from langchain.tools import StructuredTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

### 🔑 Load Environment and Initialize Watsonx.ai LLM

This section loads credentials from a local `.env` file and initializes the **Watsonx.ai large language model** (`granite-3-8b-instruct`) with custom decoding parameters.

* Uses `dotenv` to securely load API keys and endpoint info
* Instantiates the `ChatWatsonx` LLM with controlled generation settings:

  * Greedy decoding
  * Zero temperature (deterministic)
  * Up to 2000 new tokens per response

This LLM will be used throughout the notebook for reasoning and summarization.


In [59]:
load_dotenv(os.getcwd()+"/.env", override=True)

llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL"),
    apikey=os.getenv("WATSONX_API_KEY"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={
        "decoding_method": "greedy",
        "temperature": 0,
        "min_new_tokens": 5,
        "max_new_tokens": 2000
    }
)


### 🔧 Build Tool Functions: Search, Classify, and Summarize Research Papers

This section defines the core **functions** that power each tool in the agent workflow. These are the underlying implementations the agent will later use to:

* Classify the query into an arXiv research domain
* Search arXiv for recent papers in that domain
* Format and return paper details
* Summarize abstracts using Watsonx.ai LLMs

These functions are later *wrapped as tools* and made accessible to the agent.


In [60]:
from langchain.agents import Tool

# --- Tool: Search arXiv by query + category ---
def search_arxiv(query: str, category: str = "all", max_results=3):
    import requests
    import xml.etree.ElementTree as ET

    url = (
        f"http://export.arxiv.org/api/query?"
        f"search_query=cat:{category}+AND+all:{query}"
        f"&start=0&max_results={max_results}&sortBy=lastUpdatedDate"
    )
    response = requests.get(url)
    response.raise_for_status()

    root = ET.fromstring(response.text)
    ns = {"atom": "http://www.w3.org/2005/Atom"}
    papers = []

    for entry in root.findall("atom:entry", ns):
        title = entry.find("atom:title", ns).text.strip().replace("\n", " ")
        abstract = entry.find("atom:summary", ns).text.strip().replace("\n", " ")
        published = entry.find("atom:published", ns).text.strip()
        year = published[:4]

        # Look for the PDF link
        pdf_link = None
        for link in entry.findall("atom:link", ns):
            if link.attrib.get("type") == "application/pdf":
                pdf_link = link.attrib.get("href")
                break

        # Fallback to arXiv abstract page if PDF not found
        if not pdf_link:
            pdf_link = entry.find("atom:id", ns).text.strip()

        papers.append({
            "title": title,
            "abstract": abstract,
            "year": year,
            "url": pdf_link
        })

    return papers

def identify_domain_function(user_query: str) -> str:
    prompt = (
        "Classify the academic research domain of the following query. "
        "Choose one of the following: 'cs.DB' (databases), 'cs.LG' (machine learning), "
        "'cs.AI' (artificial intelligence), 'cs.CL' (natural language processing), "
        "'cs.IR' (information retrieval).\n\n"
        f"Query: {user_query}\n\nReturn only the domain code (e.g., cs.DB)."
    )
    response = llm.invoke(prompt)
    return response.content.strip()

def paper_tool_function(query: str, category: str):
    results = search_arxiv(query, category)
    formatted = []
    for i, r in enumerate(results, start=1):
        formatted.append(
            f"{i}. **Title**: {r['title']}\n"
            f"   - **Abstract**: {r['abstract']}\n"
            f"   - **Year**: {r['year']}\n"
            f"   - **PDF**: [{r['title']}]({r['url']})"
        )
    return "\n\n".join(formatted)

# --- Tool: Summarizer ---
def summarize_text_function(papers: list[dict]):
    summaries = []
    for paper in papers:
        prompt = (
            f"Title: {paper['title']}\n"
            f"Abstract: {paper['abstract']}\n"
            f"Summarize the key points in 2-3 bullets."
        )
        response = llm.invoke(prompt)
        summaries.append(
            f"**{paper['title']}**\n"
            f"{response.content}\n"
            f"🔗 [PDF Link]({paper['url']})"
        )
    return "\n\n".join(summaries)


### 🛠️ Register Tools for the Agent with Input Schemas

In this section, we wrap the previously defined functions as **LangChain `Tool` and `StructuredTool` objects**, giving the agent access to them with clear names, descriptions, and input requirements.

This setup enables the agent to:

* Use `identify_domain` to classify a topic into an arXiv category
* Use `search_papers` with structured input (`query` + `category`) to retrieve papers
* Use `summarize_abstracts` to generate concise bullet-point summaries

These tool definitions are what the agent will use to plan and execute actions during its reasoning steps.


In [61]:
# --- Input schema for search_papers tool ---
class SearchPapersInput(BaseModel):
    query: str
    category: str

# --- Tools setup ---
tools = [
    Tool(
        name="identify_domain",
        func=identify_domain_function,
        description=(
            "Use this to identify the arXiv domain (e.g., cs.DB, cs.CL, cs.LG, cs.IR) "
            "from the user's question or topic."
        )
    ),
    StructuredTool(
        name="search_papers",
        func=paper_tool_function,
        description=(
            "Use this to search arXiv for real scientific papers. "
            "Requires both 'query' and 'category' (like cs.DB)."
        ),
        args_schema=SearchPapersInput  # defined earlier with query and category
    ),
    Tool(
        name="summarize_abstracts",
        func=summarize_text_function,
        description="Use this to summarize long abstracts or paper lists in 3–5 bullet points."
    )
]

### 🧠 Define Agent Behavior with Prompt Instructions

This section creates the **system prompt** that guides the agent’s reasoning and use of tools.

The instructions define a strict workflow the agent must follow:

1. Always start by identifying the research domain
2. Then search for papers using the correct category
3. Wait for real tool responses — no guessing or fabricating
4. Summarize only when results are available
5. Only use actual data returned by tools (no hallucination)

These rules ensure the agent behaves reliably and factually when assisting with research tasks.


In [62]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
You are a research assistant that finds and summarizes real scientific papers using tools.

You MUST follow this procedure:

1. ALWAYS call `identify_domain` first to determine the arXiv category.
2. Then call `search_papers` with the topic and category.
3. Wait for the real tool response before doing anything else.
4. NEVER fabricate tool responses.
5. If the user asks for a summary, call `summarize_abstracts` using the actual abstracts from the `search_papers` result.
6. Do NOT include paper titles, authors, or abstracts unless they came directly from the tool result.

If the tool has not yet returned results, do not proceed.
"""),
    MessagesPlaceholder(variable_name="messages")
])

### 🤖 Initialize the ReAct Agent with Tools and Prompt

This step creates the **ReAct-style agent** using:

* The `llm` (Watsonx.ai model)
* The registered `tools`
* The custom `prompt` that defines how the agent should reason and act

The agent now has everything it needs to plan its steps, call tools intelligently, and respond based on real results.


In [63]:
agent_executor = create_react_agent(
    llm,
    tools,
    prompt=prompt
)

### 🗣️ Define Agent Interaction and Conversation Management Functions

This block defines two key functions for interacting with the agent and managing the conversation state:

* **`run_agent(user_prompt)`**:
  Sends a user prompt to the agent, streams the full ReAct-style reasoning trace, and prints:

  * User and assistant messages
  * Planned tool calls
  * Tool outputs
  * Token usage (if available)
  * Agent state at each step

* **`reset_conversation()`**:
  Clears the message history to restart a fresh conversation session.

These functions let you simulate and inspect multi-turn conversations with full visibility into the agent’s internal thinking.


In [64]:
conversation_messages = []

def run_agent(user_prompt: str):
    global conversation_messages

    conversation_messages.append(HumanMessage(content=user_prompt))

    state = {
    "messages": conversation_messages,
    "is_last_step": True,  # ✅ Stop after tool call
    "remaining_steps": 5
    }   

    print("\n=== Full Intermediate Execution Trace (Readable) ===")

    stream = agent_executor.stream(state, config={"verbose": True})

    for step_num, step_state in enumerate(stream):
        print(f"\n--- Step {step_num + 1} ---")

        # Extract messages from agent or tools or directly
        if "messages" in step_state:
            messages = step_state["messages"]
        elif "agent" in step_state and "messages" in step_state["agent"]:
            messages = step_state["agent"]["messages"]
        elif "tools" in step_state and "messages" in step_state["tools"]:
            messages = step_state["tools"]["messages"]
        else:
            messages = []

        for msg in messages:
            if isinstance(msg, HumanMessage):
                print(f"\n👤 User:\n{msg.content}")

            elif isinstance(msg, AIMessage):
                content = msg.content.strip() or "[No assistant content]"
                print(f"\n🤖 Assistant:\n{content}")

                tool_calls = msg.additional_kwargs.get("tool_calls", [])
                for call in tool_calls:
                    fn_name = call["function"]["name"]
                    args = call["function"]["arguments"]
                    print(f"🧠 Tool Call Planned → {fn_name} with args: {args}")

                if "token_usage" in msg.response_metadata:
                    usage = msg.response_metadata["token_usage"]
                    print(f"📊 Token usage: prompt={usage['prompt_tokens']}, completion={usage['completion_tokens']}, total={usage['total_tokens']}")

                conversation_messages.append(msg)

            elif isinstance(msg, ToolMessage):
                print(f"\n🔧 Tool Output ({msg.name}):\n{msg.content[:800]}")
                if len(msg.content) > 800:
                    print("... [truncated]")
                conversation_messages.append(msg)

        # Debug other non-message state keys if needed
        other_keys = set(step_state.keys()) - {"messages", "agent", "tools"}
        for key in other_keys:
            print(f"\n🔍 State[{key}]: {step_state[key]}")
            
def reset_conversation():
    global conversation_messages
    conversation_messages = []
    print("🔄 Conversation reset.")

### ▶️ Call the Agent

In [65]:
reset_conversation()
run_agent("Find recent 3 papers on database query plan representation using transformers since 2024")

🔄 Conversation reset.

=== Full Intermediate Execution Trace (Readable) ===

--- Step 1 ---

🤖 Assistant:
[No assistant content]
🧠 Tool Call Planned → search_papers with args: {"query": "database query plan representation transformers", "category": "cs.DB"}
📊 Token usage: prompt=555, completion=36, total=591

--- Step 2 ---

🔧 Tool Output (search_papers):
1. **Title**: SQL-Factory: A Multi-Agent Framework for High-Quality and Large-Scale   SQL Generation
   - **Abstract**: High quality SQL corpus is essential for intelligent database. For example, Text-to-SQL requires SQL queries and correspond natural language questions as training samples. However, collecting such query corpus remains challenging in practice due to the high cost of manual annotation, which highlights the importance of automatic SQL generation. Despite recent advances, existing generation methods still face limitations in achieving both diversity and cost-effectiveness. Besides, many methods also treat all tables equa

In [57]:
run_agent("Can you summarize the papers from search?")


=== Full Intermediate Execution Trace (Readable) ===

--- Step 1 ---

🤖 Assistant:
Here's a summary of the three papers:

1. **SQL-Factory: A Multi-Agent Framework for High-Quality and Large-Scale SQL Generation** (2025)
   - The paper proposes a multi-agent framework, SQL-Factory, for high-quality and large-scale SQL generation. It decomposes the generation process into three collaborative teams: Generation Team, Expansion Team, and Management Team. The framework ensures a balanced trade-off between diversity, scalability, and generation cost. SQL-Factory generates over 300,000 SQL queries with less than $200 API cost, achieving higher diversity compared to other methods and significantly improving model performance in various downstream tasks.

2. **S3AND: Efficient Subgraph Similarity Search Under Aggregated Neighbor Difference Semantics (Technical Report)** (2025)
   - This paper introduces the problem of subgraph similarity search under aggregated neighbor difference semantics (S